# Narnian DEMO

## Table of contents
1. [Donwload Repo](#download-repo)
2. [Install Dependencies](#install-dependencies)
3. [1st example](#1st-example)
4. [2nd example](#2nd-example)
5. [3rd example](#3rd-example)

### Download Repo <a name="download-repo"></a>

In [1]:
oauth_token = input("Insert github oauth token: ")

In [ ]:
%git clone https://github.com/mela64/narnian.git

### Install Dependencies <a name="install-dependencies"></a>

In [ ]:
%cd narnian/
%pip install -r -q requirements.txt

### Load Dependencies


In [3]:
import torch
from narnian.model import Model
from narnian.attributes import Attributes
from modules.networks import PredRNN
from modules.networks import GenCTBEInitStateBZeroInput, GenRNN
from narnian.server import Server
from narnian.streams import Stream
from narnian.model import EmptyModel
from basic.basic_agent import BasicAgent
from basic.basic_streams import Sin, Square
from basic.basic_environment import BasicEnvironment

### Define the Models used in the Demo

Demo models consists practically of a **generator** and a **predictor**:
- Generator: in this case, we have 2 kinds of generators: 
    - `GenRNN`: A simple rnn model (**DemoModel1**).
    - `GenCTBE..`: An antysimmetric rnn with Exact Matrix Exponential Blocks (**DemoModel2**).
- Predictor: We will have a case with and without the predictor:
    - `None`: For this model no predictor is defined (**DemoModel1**).
    - `PredRNN`: A simple rnn model (**DemoModel2**).

Along the models definition, DemoModel includes also the `learn` method which, in this specific case, it is implemented as a common GD training step.

In [4]:
class DemoModel1(Model):

    def __init__(self, attributes: list[Attributes], lr: float = 0.0001, device: torch.device = torch.device("cpu")):
        """Creates a model composed of a generator and a predictor."""

        # getting shape info from attributes (it is needed to build the generator/predictor)
        assert len(attributes) == 2, "Only two attributes are supported/expected (about y and d)"
        u_shape = attributes[0].shape
        d_dim = attributes[1].shape.numel()
        y_dim = attributes[0].shape.numel()
        
        generator = GenRNN(u_shape=u_shape, d_dim=d_dim, y_dim=y_dim, h_dim=150)
        # generator = GenCTBEInitStateBZeroInput(u_shape=u_shape, d_dim=d_dim, y_dim=y_dim, h_dim=150, delta=0.1,
        #                                        local=True, cnu_memories=0)
        predictor = None

        # creating the model (superclass)
        super(DemoModel1, self).__init__(generator, predictor, attributes, device=device)

        # extra stuff
        self.optim = torch.optim.SGD(list(self.generator.parameters()), lr=lr)
        self.loss_gen = torch.nn.functional.mse_loss
        self.loss_pred = torch.nn.functional.mse_loss

    def learn(self,
              y: torch.Tensor | None, yhat: torch.Tensor | None,
              d: torch.Tensor | None, dhat: torch.Tensor | None) \
            -> tuple[float, torch.Tensor | None, torch.Tensor | None, torch.Tensor | None, torch.Tensor | None]:
        """Learn from different types of data (some of them could be None)."""

        # clean arguments, ensuring that what should be forced to None is actually forced to None
        _, y, yhat, d, dhat = super().learn(y, yhat, d, dhat)  # it seems unuseful, but IT MUST be called!

        # evaluating loss function
        loss = ((self.loss_gen(y, yhat) if yhat is not None else 0.) +
                (self.loss_pred(d, dhat) if dhat is not None else 0.))

        # learning
        self.optim.zero_grad()
        loss_as_float = loss.item()
        loss.backward()
        self.optim.step()

        return loss_as_float, y, yhat, d, dhat

In [ ]:
class DemoModel2(Model):

    def __init__(self, attributes: list[Attributes], lr: float = 0.0001, device: torch.device = torch.device("cpu")):
        """Creates a model composed of a generator and a predictor."""

        # getting shape info from attributes (it is needed to build the generator/predictor)
        assert len(attributes) == 2, "Only two attributes are supported/expected (about y and d)"
        u_shape = attributes[0].shape
        d_dim = attributes[1].shape.numel()
        y_dim = attributes[0].shape.numel()

        generator = GenCTBEInitStateBZeroInput(u_shape=u_shape, d_dim=d_dim, y_dim=y_dim, h_dim=500, delta=0.1,
                                               local=False, cnu_memories=0)
        predictor = PredRNN(y_dim=y_dim, d_dim=d_dim, h_dim=10)

        # creating the model (superclass)
        super(DemoModel2, self).__init__(generator, predictor, attributes, device=device)

        # extra stuff
        self.optim = torch.optim.SGD(list(self.generator.parameters()) + list(self.predictor.parameters()), lr=lr)
        self.loss_gen = torch.nn.functional.mse_loss
        self.loss_pred = torch.nn.functional.mse_loss

    def learn(self,
              y: torch.Tensor | None, yhat: torch.Tensor | None,
              d: torch.Tensor | None, dhat: torch.Tensor | None) \
            -> tuple[float, torch.Tensor | None, torch.Tensor | None, torch.Tensor | None, torch.Tensor | None]:
        """Learn from different types of data (some of them could be None)."""

        # clean arguments, ensuring that what should be forced to None is actually forced to None
        _, y, yhat, d, dhat = super().learn(y, yhat, d, dhat)  # it seems unuseful, but IT MUST be called!

        # evaluating loss function
        loss = ((self.loss_gen(y, yhat) if yhat is not None else 0.) +
                (self.loss_pred(d, dhat) if dhat is not None else 0.))

        # learning
        self.optim.zero_grad()
        loss_as_float = loss.item()
        loss.backward()
        self.optim.step()

        return loss_as_float, y, yhat, d, dhat


## Demo 1: Simple Teacher-Student with one stream to learn

We decided to name it 'env' with a certain title 'Demo Sandbox Signal'

In [5]:
# creating environment
env = BasicEnvironment("env", title="Demo Sandbox Signal")
device = torch.device("cpu")

Then we define a new stream which is a sin wave. 
A stream is characterized by a name, the source where the stream is coming from (the enviroment in this case), the stream itself .

In [6]:
env.add_stream(Stream.create(name="sin", creator=env.name, stream=Sin(freq=0.06, phase=0.5, delta=0.1)))

Now it's time for defining the FSMs for the agents and the enviroment.
Lets start with the FSM of the enviroment...

What we want is to:
  1. Moving from the init state to letting the streams of the enviroment enabled.
  2. Since the streams are enabled, we can now letting all the agents know that such streams are available.
  3. The enviroment will then move to the 'ready' state, making all the agents knowing each other.

In [7]:
# modeling behaviour of the environment
env.add_transit("init", "streams_enabled", action="enable_all_streams")
env.add_transit("streams_enabled", "streams_sent", action="send_streams_to_all")
env.add_transit("streams_sent", "ready", action="send_agents_to_all")

#### Now we can define the Teacher agent

We define the agent that will act the role of the Teacher in this demo.

Since each agent is characterized by a name, a model and its autority within the environment, we can now define the teacher agent as follows:
- Name: 'Teacher'
- Model: A dummy model 
- Authority: 1 (the highest)

In [8]:
ag = BasicAgent("Teacher", model=EmptyModel(), authority=1.0)


Define the FSM of the teacher agent...

What we want is to:
  1. Let the teacher agent know that the streams are now available. (init -> got_streams) _Rember that in the enviroment FSM we have enabled the streams_


In [9]:
ag.add_transit("init", "got_streams", action="get_streams")

  2. The teacher, after knowing the streams, is waiting for knowing the other agents. 

In [10]:
ag.add_transit("got_streams", "got_agents", action="get_agents")

  3. The teacher, after knowing the other agents, is ready to record the streams flowing in the enviroment.

To identify the streams, we can use the `stram_hash` attribute, in this specific case the associated hash is "env:sin"
We decide to record 2000 samples of the stream.

In [11]:
ag.add_transit("got_agents", "recording1", action="record", args={"stream_hash": env.name + ":sin", "steps": 2000})


The teacher, after recording the streams, start to find for agents to engage.
_(Since this action does not imply any change in the FSM, we add a state action with the state recording1)_

`min_auth` and the `max_auth` are set to 0.0, meaning that we are waiting for agent with autority strictly equal to 0

In [12]:
ag.add_state_action("recording1", action="find_agent_to_engage", args={"min_auth": 0.0, "max_auth": 0.0})

  4. When students are found the teacher starts an engagement with them.

In [13]:
ag.add_transit("recording1", "student_found", action="send_engagement")

5. All the students which answer with a positive feedback are then engaged in the learning process.

In [14]:
ag.add_transit("student_found", "student_engaged", action="got_engagement")


6. The streams are shared with the students.

In [15]:
ag.add_transit("student_engaged", "stream_shared", action="share_streams")


7. The teacher asks the students to learn the function (in this case the sin).

We set the input-output/descriptor stream to the stream hash of the teacher.
We ask to learn for 2000 steps

In [16]:
ag.add_transit("stream_shared", "asked_learn", action="ask_learn_gen",
               args={"du_hash": ag.name + ":recorded1", "yhat_hash": ag.name + ":recorded1", "dhat_hash": ag.name + ":recorded1", "ask_steps": 2000})

8. We end up the learning stage and we ask the students to generate the function.

In [17]:
ag.add_transit("asked_learn", "done_learn", action="done_learn_gen")
ag.add_transit("done_learn", "asked_gen", action="ask_gen", args={"du_hash": ag.name + ":record1", "ask_steps": 2000})

9. We end up the process and move to the end state.

In [18]:
ag.add_transit("asked_gen", "finished", action="done_gen")


At this point we can add the Teacher to the environment.

In [19]:
# adding agent to environment
env.add_agent(ag)

#### Now we can define a student agent

We define the agent that will act the role of the Student in this demo.

In [20]:
ag = BasicAgent("Student", model=DemoModel1(attributes=env.shared_attributes, lr=0.001, device=device), authority=0.0)

We now set its FSM...

1. The student starts by acquiring the information of what streams are circulating in the enviroment.
2. The student knows who are the other agents in the environment.
3. The student is ready to engage with a teacher (notice now that we are looking for `min_auth` and `max_auth` equal to 1).
4. After the engagement with a teacher, the student got information about the streams available for such teacher.
5. The student is ready to learn the function, and starts the learning procedure.
6. The student is ready to generate the function, and generate it. 

In [21]:
# creating student FSM
ag.add_transit("init", "got_streams", action="get_streams")
ag.add_transit("got_streams", "got_agents", action="get_agents")
ag.add_transit("got_agents", "teacher_engaged", action="get_engagement", args={"min_auth": 1.0, "max_auth": 1.0})
ag.add_transit("teacher_engaged", "got_teacher_streams", action="get_streams")
ag.add_transit("got_teacher_streams", "learning", action="do_learn_gen")
ag.add_transit("got_teacher_streams", "generated", action="do_gen")
ag.add_transit("learning", "got_teacher_streams", action="nop")
ag.add_transit("generated", "got_teacher_streams", action="nop")


# adding agent to environment
env.add_agent(ag)


Add a server to let the enviroment be accesible with a web app

In [23]:
Server(env=env, port=2001)

 * Serving Flask app 'narnian.server'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:2001
 * Running on http://10.1.0.72:2001
Press CTRL+C to quit


10.1.0.119 - - [20/Mar/2025 12:48:39] "GET / HTTP/1.1" 200 -
10.1.0.119 - - [20/Mar/2025 12:48:39] "GET /index_dist.css HTTP/1.1" 200 -
10.1.0.119 - - [20/Mar/2025 12:48:39] "GET /static/css/main.752eb79b.css HTTP/1.1" 200 -
10.1.0.119 - - [20/Mar/2025 12:48:39] "GET /static/js/main.6d72e55a.js HTTP/1.1" 200 -
10.1.0.119 - - [20/Mar/2025 12:48:40] "GET /get_env_name HTTP/1.1" 200 -
10.1.0.119 - - [20/Mar/2025 12:48:40] "GET /favicon.ico HTTP/1.1" 200 -
10.1.0.119 - - [20/Mar/2025 12:48:40] "GET /get_list_of_agents?agent_name=env HTTP/1.1" 200 -
[2025-03-20 12:48:40,291] ERROR in app: Exception on /get_play_pause_status [GET]
Traceback (most recent call last):
  File "/home2/dimaio/miniconda3/envs/continuallm/lib/python3.10/site-packages/flask/app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
  File "/home2/dimaio/miniconda3/envs/continuallm/lib/python3.10/site-packages/flask/app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
  F

In [ ]:
# RUN THE ENVIROMENT
env.run()

[envir: env] >>> Clock step 0 <<<
[envir: env, state: init, act: enable_all_streams()] Enabling all streams
[envir: env] >>> Clock step 1 <<<
[envir: env, state: streams_enabled, act: send_streams_to_all()] Providing 1 streams to all agents
[agent: Teacher, state: init, act: get_streams(agent=None, streams={...})] Getting 1 streams from the environment
[agent: Student, state: init, act: get_streams(agent=None, streams={...})] Getting 1 streams from the environment
[envir: env] >>> Clock step 2 <<<
[envir: env, state: streams_sent, act: send_agents_to_all()] Sharing contacts of 2 agents with all the other agents
[agent: Teacher, state: got_streams, act: get_agents(agent=None, agents={...})] Getting contacts of 2 agents from the environment
[agent: Student, state: got_streams, act: get_agents(agent=None, agents={...})] Getting contacts of 2 agents from the environment
[envir: env] >>> Clock step 3 <<<
[agent: Teacher, state: got_agents, act: record(stream_hash='env:sin', steps=2000)] Rec

## Demo 2: learn and generate a playlist
In this case, we are creating a Sandbox in which the Teacher has 2 different stream samples available for its Students.\
The lesson is structured as a playlist in which the Teacher provides the supervision, alternating the two signals.\
After 5 repetitions of each signal, the Student is aked to reproduce both of them.

In [ ]:
# creating environment
env = BasicEnvironment("env", title="Demo Sandbox Playlist")
device = torch.device("cpu")

# adding streams to the environment
env.add_stream(Stream.create(name="sin", creator=env.name, stream=Sin(freq=0.06, phase=0.5, delta=0.1)))
env.add_stream(Stream.create(name="square", creator=env.name, stream=Square(freq=0.06, ampl=0.5, phase=0.5, delta=0.1)))

# modeling behaviour of the environment
env.add_transit("init", "streams_enabled", action="enable_all_streams")
env.add_transit("streams_enabled", "streams_sent", action="send_streams_to_all")
env.add_transit("streams_sent", "ready", action="send_agents_to_all")

# creating teacher agent
ag = BasicAgent("Teacher", model=EmptyModel(), authority=1.0)
ag.add_transit("init", "got_streams", action="get_streams")
ag.add_transit("got_streams", "got_agents", action="get_agents")
ag.add_transit("got_agents", "recording1", action="record", args={"stream_hash": env.name + ":sin", "steps": 500})
ag.add_transit("recording1", "recording2", action="record", args={"stream_hash": env.name + ":square", "steps": 500})
ag.add_transit("recording2", "playlist_ready", action="set_pref_streams",
               args={"stream_hashes": [ag.name + ":recorded1", ag.name + ":recorded2"], "repeat": 6})
ag.add_state_action("playlist_ready", action="find_agent_to_engage", args={"min_auth": 0.0, "max_auth": 0.0})
ag.add_transit("playlist_ready", "student_found", action="send_engagement")
ag.add_transit("student_found", "playlist_ready", action="nop")
ag.add_transit("student_found", "student_engaged", action="got_engagement")
ag.add_transit("student_engaged", "stream_shared", action="share_streams")
ag.add_transit("stream_shared", "asked_learn", action="ask_learn_gen",
               args={"du_hash": "<playlist>", "yhat_hash": "<playlist>", "dhat_hash": "<playlist>",
                     "ask_steps": 500})
ag.add_transit("asked_learn", "done_learn", action="done_learn_gen")
ag.behav.add_state_action("done_learn", action="next_pref_stream")
ag.behav.add_transit("done_learn", "stream_shared", action="check_pref_stream", args={"what": "not_last_round"})
ag.behav.add_transit("done_learn", "ready_to_ask", action="check_pref_stream", args={"what": "last_round"})
# add a final unsupervised generation for each signal
ag.behav.add_transit("ready_to_ask", "asked_gen", action="ask_gen",
                     args={"du_hash": "<playlist>",  "dhat_hash": "<playlist>", "ask_steps": 500})
ag.behav.add_transit("asked_gen", "done_gen", action="done_gen")
ag.behav.add_state_action("done_gen", action="next_pref_stream")
ag.behav.add_transit("done_gen", "ready_to_ask", action="check_pref_stream", args={"what": "not_first"})
ag.behav.add_transit("done_gen", "finished", action="check_pref_stream", args={"what": "first"})

# adding agent to environment
env.add_agent(ag)

# creating student agent
ag = BasicAgent("Student", model=BasicModel(attributes=env.shared_attributes, lr=0.001, device=device),
                authority=0.0)
ag.add_transit("init", "got_streams", action="get_streams")
ag.add_transit("got_streams", "got_agents", action="get_agents")
ag.add_transit("got_agents", "teacher_engaged", action="get_engagement", args={"min_auth": 1.0, "max_auth": 1.0})
ag.add_transit("teacher_engaged", "got_teacher_streams", action="get_streams")
ag.add_transit("got_teacher_streams", "learning", action="do_learn_gen_and_pred")
ag.add_transit("got_teacher_streams", "generated", action="do_gen_and_pred")
ag.add_transit("learning", "got_teacher_streams", action="nop")
ag.add_transit("generated", "got_teacher_streams", action="nop")

# adding agent to environment
env.add_agent(ag)

# printing
print(env)
for ag in env.agents.values():
    print(ag)

# creating server
Server(env=env)

# running
env.run()


## learn, generate playlist and descriptor

## old python code

#### basic_model_gen4all_pre4all_gd

In [ ]:
class DemoModel(Model):

    def __init__(self, attributes: list[Attributes], lr: float = 0.0001, device: torch.device = torch.device("cpu")):
        """Creates a model composed of a generator and a predictor."""

        # getting shape info from attributes (it is needed to build the generator/predictor)
        assert len(attributes) == 2, "Only two attributes are supported/expected (about y and d)"
        u_shape = attributes[0].shape
        d_dim = attributes[1].shape.numel()
        y_dim = attributes[0].shape.numel()

        generator = GenCTBEInitStateBZeroInput(u_shape=u_shape, d_dim=d_dim, y_dim=y_dim, h_dim=500, delta=0.1,
                                               local=False, cnu_memories=0)
        predictor = PredRNN(y_dim=y_dim, d_dim=d_dim, h_dim=10)

        # creating the model (superclass)
        super(BasicModel, self).__init__(generator, predictor, attributes, device=device)

        # extra stuff
        self.optim = torch.optim.SGD(list(self.generator.parameters()) + list(self.predictor.parameters()), lr=lr)
        self.loss_gen = torch.nn.functional.mse_loss
        self.loss_pred = torch.nn.functional.mse_loss

    def learn(self,
              y: torch.Tensor | None, yhat: torch.Tensor | None,
              d: torch.Tensor | None, dhat: torch.Tensor | None) \
            -> tuple[float, torch.Tensor | None, torch.Tensor | None, torch.Tensor | None, torch.Tensor | None]:
        """Learn from different types of data (some of them could be None)."""

        # clean arguments, ensuring that what should be forced to None is actually forced to None
        _, y, yhat, d, dhat = super().learn(y, yhat, d, dhat)  # it seems unuseful, but IT MUST be called!

        # evaluating loss function
        loss = ((self.loss_gen(y, yhat) if yhat is not None else 0.) +
                (self.loss_pred(d, dhat) if dhat is not None else 0.))

        # learning
        self.optim.zero_grad()
        loss_as_float = loss.item()
        loss.backward()
        self.optim.step()

        return loss_as_float, y, yhat, d, dhat


#### sandbox_example

In [ ]:
import torch
from narnian.server import Server
from narnian.streams import Stream
from narnian.model import EmptyModel
from basic.basic_agent import BasicAgent
from basic.basic_model_gen4all_pred4all_gd import BasicModel
from basic.basic_streams import Sin, Square
from basic.basic_environment import BasicEnvironment

# creating environment
env = BasicEnvironment("env", title="Sample Sandbox")
device = torch.device("cpu")

# adding streams to the environment
env.add_stream(Stream.create(name="sin", creator=env.name, stream=Sin(freq=0.06, phase=0.5, delta=0.1)))
env.add_stream(Stream.create(name="square", creator=env.name, stream=Square(freq=0.06, ampl=0.5, phase=0.5, delta=0.1)))

# modeling behaviour of the environment
env.add_transit("init", "streams_enabled", action="enable_all_streams")
env.add_transit("streams_enabled", "streams_sent", action="send_streams_to_all")
env.add_transit("streams_sent", "ready", action="send_agents_to_all")

# creating teacher agent
ag = BasicAgent("Teacher", model=EmptyModel(), authority=1.0)
ag.add_transit("init", "got_streams", action="get_streams")
ag.add_transit("got_streams", "got_agents", action="get_agents")
ag.add_transit("got_agents", "recording1", action="record",
               args={"stream_hash": env.name + ":sin", "steps": 500})
ag.add_transit("recording1", "recording2", action="record",
               args={"stream_hash": env.name + ":square", "steps": 500})
ag.add_transit("recording2", "playlist_ready", action="set_pref_streams",
               args={"stream_hashes": [ag.name + ":recorded1", ag.name + ":recorded2"], "repeat": 1})
ag.add_state_action("playlist_ready", action="find_agent_to_engage", args={"min_auth": 0.0, "max_auth": 0.0})
ag.add_transit("playlist_ready", "student_found", action="send_engagement")
ag.add_transit("student_found", "playlist_ready", action="nop")
ag.add_transit("student_found", "student_engaged", action="got_engagement")
ag.add_transit("student_engaged", "stream_shared", action="share_streams")
ag.add_transit("stream_shared", "asked_learn", action="ask_learn_gen_and_pred",
               args={"du_hash": "<playlist>", "yhat_hash": "<playlist>", "dhat_hash": "<playlist>",
                     "ask_steps": 500})
ag.add_transit("asked_learn", "done_learn", action="done_learn_gen_and_pred")
ag.add_transit("done_learn", "asked_gen", action="ask_gen_and_pred",
               args={"du_hash": "<playlist>", "ask_steps": 500})
ag.add_transit("asked_gen", "done_gen", action="done_gen_and_pred")
ag.add_state_action("done_gen", action="eval",
                    args={"stream_hash": "<playlist>", "what": "y", "how": "mse", "steps": 500})
ag.add_transit("done_gen", "stream_shared", action="compare_eval", args={"cmp": ">", "thres": 0.05})
ag.add_transit("done_gen", "good", action="compare_eval", args={"cmp": "<=", "thres": 0.05})
ag.add_state_action("good", action="next_pref_stream")
ag.add_transit("good", "stream_shared", action="check_pref_stream", args={"what": "not_first"})
ag.add_transit("good", "finished", action="check_pref_stream", args={"what": "first"})

# adding agent to environment
env.add_agent(ag)

# creating student agent
ag = BasicAgent("Student", model=BasicModel(attributes=env.shared_attributes, lr=0.001, device=device),
                authority=0.0)
ag.add_transit("init", "got_streams", action="get_streams")
ag.add_transit("got_streams", "got_agents", action="get_agents")
ag.add_transit("got_agents", "teacher_engaged", action="get_engagement", args={"min_auth": 1.0, "max_auth": 1.0})
ag.add_transit("teacher_engaged", "got_teacher_streams", action="get_streams")
ag.add_transit("got_teacher_streams", "learning", action="do_learn_gen_and_pred")
ag.add_transit("got_teacher_streams", "generated", action="do_gen_and_pred")
ag.add_transit("learning", "got_teacher_streams", action="nop")
ag.add_transit("generated", "got_teacher_streams", action="nop")

# adding agent to environment
env.add_agent(ag)

# printing
print(env)
for ag in env.agents.values():
    print(ag)

# creating server
Server(env=env)

# running
env.run()
